In [ ]:
import numpy as np

print("输入参评数目：")
n = int(input())

print("输入参评指标数目：")
m = int(input())

print("输入矩阵类型：1：极大型 2：极小型 3. 中间型 4. 区间型")
kind= input().split(" ")

print("输入矩阵：")
A = np.zeros((n,m))

#每次输入一行
for i in range(n):
    A[i] = input().split(" ")
    A[i] = list(map(float,A[i]))
        
print("输入矩阵为：\n{}".format(A))

![图片1](TOPSIS_01.png)

In [5]:
#极小型指标转换为极大型指标

def minToMax(maxx, x):
    x = list(x)
    ans = [[maxx-e] for e in x]
    return np.array(ans)

In [ ]:
#中间型指标转换为极大型指标

def midToMax(bestx, x):
    x = list(x)
    h = [abs(e - bestx) for e in x]
    M = max(h)
    if M == 0:
        M = 1 #防止最大差值为0
        
    ans = [[(1-e/M)] for e in h] #计算每个差值占最大差值的比例，并从1中减去，得到新指标值
    
    return np.array(ans)


In [ ]:
#区间型指标转换为极大型指标

def ragToMax(lowx, highx, x):
    x = list(x)
    M = max(lowx-min(x), max(x)-highx)
    if M==0:
        M = 1 #防止最大距离为0
        
    ans = []
    for i in range(len(x)):
        if x[i] < lowx:
            ans.append(1-(lowx-x[i])/M)#如果指标值小于下限，则新指标值等于1减去指标值占最大距离的比例
        elif x[i] > highx:
            ans.append(1-(x[i]-highx)/M)#如果指标值大于上限，则新指标值等于指标值占最大距离的比例
        else:
            ans.append([1])#指标在区间内，新指标值等于1
            
    return np.array(ans)


In [ ]:
#统一指标类型，全部转化为极大型指标

X = np.zeros((n,1))
for i in range(n):
    if kind[i] == "1": #极大型指标
        v = np.array(A[:,i]) #切片，提取所有行的第i列
        
    elif kind[i] == "2": #极小型指标
        maxA = max(A[:,i])
        v = minToMax(maxA, A[:,i])
        
    elif kind[i] == "3": #中间型指标
        print("请输入最优值：")
        bestA = eval(input())
        v = midToMax(bestA, A[:,i])
        
    elif kind[i] == "4": #区间型指标
        print("请输入下限值：")
        lowA = eval(input())
        print("请输入上限值：")
        highA = eval(input())
        v = ragToMax(lowA, highA, A[:,i])
        
    if i == 0:
        X = v.reshape(-1, 1) #如果是第一行，则直接赋值
    else:
        X = np.hstack((X,v.reshape(-1, 1))) #否则，水平拼接
        
    print("统一后的矩阵为：\n{}".format(X))
    

![图片2](TOPSIS_02.png)

In [ ]:
#对统一指标后的矩阵进行标准化处理

X= X.astype('float')
for j in range(n):
    X[:,j] = X[:,j]/np.sqrt(sum(X[:,j]**2))

print("标准化后的矩阵为：\n{}".format(X))

![图片3](TOPSIS_03.png)

In [ ]:
#最大最小值距离的计算

x_max = np.max(X, axis=0)
x_min = np.min(X, axis=0)
d_z = np.sqrt(np.sum(np.square((X - np.tile(x_max,(n,1)))),axis=1)) #计算每个参评对象与最优情况的距离d+
d_f = np.sqrt(np.sum(np.square((X - np.tile(x_min,(n,1)))),axis=1)) #计算每个参评对象与最差情况的距离d-

print("每个指标最大值：", x_max)
print("每个指标最小值：", x_min)
print("每个参评对象与最优情况的距离d+：", d_z)
print("每个参评对象与最差情况的距离d-：", d_f)


In [6]:
#计算每个参评对象得分排名
s = d_f/(d_z*d_f)
Score = 100*s/sum(s)
for i in range(len(Score)):
    print(f"第{i+1}个标准化后百分制得分为{Score[i]}")